In [17]:
import numpy as np

In [18]:
pi = np.pi

## Algorithm 1

Given an $N \times N$ matrix $A$ denoted as $\{ a \} _{i,j = 1}^{N,N}$ the routine replaces it by the LU decomposition of a row-wise permutation of itself. $"a"$ and $"N"$ are input. $"a"$ is also output, modified to apply the LU decomposition; $\{ indx_i \} _{i,j = 1}^N$ is an output vector that records the row permutation affected by the partial pivoting; $"d"$ is output and adopts $\pm 1 $ depending on whether the number of row interchanges was even or off. This routine is used in combination with algorithm 2 to solve linear equations or invert a matrix.

In [35]:
def ludecmp(A, N):

    #this function performs LU decomposition of a matrix A

    tiny = 1e-20

    d = 1.  

    vv = np.zeros(N)
    indx = np.zeros(N)

    for ii in range(0,N): # python index starts at i = 0, range does not include N

        a_max = 0. # big = 0

        for jj in range(0,N):  

            a_ij = np.abs(A[ii,jj])  #temp = abs(a_ij)

            if (a_ij > a_max):
                a_max = a_ij


        # check if matrix is singular; exit if singular    
        if (a_max == 0.):
                print("This matrix is singular")
                exit

        vv[ii] = 1./a_max
        

    for jj in range(0,N):

        for ii in range(0,jj-1):

            sums = A[ii,jj]

            for kk in range(0,ii-1):

                sums = sums - A[ii,kk] * A[kk, jj]
            
            A[ii,jj] = sums

        a_max = 0.

        for ii in range(jj,N): 
            sums = A[ii,jj]

            for kk in range(0,jj-1):

                sums = sums - A[ii,kk] * A[kk, jj]

            A[ii,jj] = sums

            dummy = vv[ii] * np.abs(A[ii,jj])

            if (dummy >= a_max):
                a_max = dummy

                i_max = ii


        if (jj != i_max):

            for kk in range(0,N):

                dummy = A[i_max,kk]

                A[i_max,kk] = A[jj,kk]

                A[jj,kk] = dummy


            d = -d

            vv[i_max] = vv[jj]

        indx[jj] = i_max

            # account for division by zero

        if (A[jj,jj] == 0.):
            A[jj,jj] = tiny

        if (jj != N):
            dummy = 1/A[jj,jj]

            for ii in range(jj, N): ## indexing might just be jj not jj +1
                A[ii,jj] = A[ii,jj] * dummy

    print(A)
    return indx, d, A


In [36]:
# construct A matrix
A =np.matrix([[1, 2, -1], [6, -5, 4] , [-9, 8, -7]])
b = np.array([2*pi, 5*pi,-8*pi])

# Store size N for any N x N matrix

N = np.shape(A)[0]

indx, d, A = ludecmp(A,N)


[[ 1  8 -7]
 [ 0  1 -1]
 [ 0 -2  1]]


## Algorithm 2

Solve the set of $N$ linear equations $\textbf{A} \vec{x} = \vec{b}$. Matrix $\{ a \} _{i,j = 1}^{N,N}$ is the LU decomposition of original matrix $A$ obtained from Algorithm 1. Vector $\{ indx_i \} _{i,j = 1}^N$ is input as the permutation vector returned by Algorithm 1. Vector $\{b_i\} _{i=1}^N$ is input as the right hand side vector $\vec{b}$ but returns with solution vector $\vec{x}$. Inputs $\{ a \} _{i,j = 1}^{N,N}$, $N$ and $\{ indx_i \} _{i,j = 1}^N$ 

In [21]:
def lubksb(b,A,N,indx):
    ii = 0
    for i in range (0,N):
        ip = indx[i]
        sums = b[ip]
        b[ip] = b[i]

        if (ii != 0.):
            for jj in range(ii, i-1):
                sums = sums - a[i,jj] * b[jj]
        elif (sums != 0.):
            ii = i
        
    b[i] = sums

    for i in range(N-1,0,-1):
        sums = b[i]
        for jj in range (i,N):
            sums = sums - a[i,jj] * b[jj]

        b[i] = sums/a[ii,ii]
        
    return b

In [22]:
# construct A matrix
A =np.matrix([[1, 2, -1], [6, -5, 4] , [-9, 8, -7]])
b = np.array([2*pi, 5*pi,-8*pi])
# Store size N for any N x N matrix

N = np.shape(A)[0]

indx, d, A = ludecmp(A,N)


[[ 1  8 -7]
 [ 0  1 -1]
 [ 0 -2  1]]
